In [2]:
import torch
from datasets import load_from_disk

In [3]:
dataset = load_from_disk("tokenised_wmt14")

In [4]:
print(dataset["train"][0])

{'input_ids': [6379, 19198, 306, 277, 7688], 'attention_mask': [1, 1, 1, 1, 1], 'labels': [16144, 15612, 321, 15617]}
